# Análisis RFM - Segmentación de Clientes

Este notebook realiza un análisis RFM (Recency, Frequency, Monetary) sobre los datos de ventas para segmentar a los clientes según su comportamiento de compra:
- **Recencia**: ¿Qué tan reciente fue la última compra del cliente?
- **Frecuencia**: ¿Con qué frecuencia compra?
- **Valor Monetario**: ¿Cuánto dinero gasta?

Estas tres métricas ayudan a identificar clientes de alto valor, clientes en riesgo de abandono y otros segmentos importantes para la estrategia comercial.

In [ ]:
# Instalación de las librerías necesarias
!pip install pandas numpy matplotlib seaborn plotly

In [ ]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

## 1. Carga y Preparación de Datos

In [ ]:
# Cargar los datos de ventas descargando primero los bytes sin procesar y luego decodificando adecuadamente
import requests
import io

url = "https://pocs.nyc3.cdn.digitaloceanspaces.com/sales_data_sample.csv"

# Descargar los bytes sin procesar
response = requests.get(url)
raw_data = response.content

# Primero, intentar detectar la codificación - probablemente latin1 o cp1252 basado en el error
try:
    decoded_content = raw_data.decode('latin1')
    encoding_used = 'latin1'
except UnicodeDecodeError:
    try:
        decoded_content = raw_data.decode('cp1252')
        encoding_used = 'cp1252'
    except UnicodeDecodeError:
        # Alternativa a una codificación muy permisiva que raramente falla
        decoded_content = raw_data.decode('latin1', errors='replace')
        encoding_used = 'latin1 with replacement'

print(f"Archivo decodificado exitosamente usando codificación {encoding_used}")

# Cargar el CSV desde el string decodificado correctamente
ventas = pd.read_csv(io.StringIO(decoded_content))

# Mostrar las primeras filas
ventas.head()

In [ ]:
# Verificar la forma y la información de los datos
print(f"Forma del dataset: {ventas.shape}")
print("\nTipos de datos:")
ventas.info()

In [ ]:
# Verificar valores faltantes
valores_faltantes = ventas.isnull().sum()
print("Valores faltantes:")
print(valores_faltantes[valores_faltantes > 0] if len(valores_faltantes[valores_faltantes > 0]) > 0 else "No hay valores faltantes")

In [ ]:
# Convertir ORDERDATE a formato datetime
ventas['ORDERDATE'] = pd.to_datetime(ventas['ORDERDATE'])

# Filtrar pedidos cancelados si existen
ventas = ventas[ventas['STATUS'] != 'Cancelled']

## 2. Cálculo de Métricas RFM

In [ ]:
# Obtener la fecha más reciente en el dataset
fecha_max = ventas['ORDERDATE'].max()
print(f"Fecha de pedido más reciente: {fecha_max}")

# Crear fecha de análisis (1 día después del pedido más reciente)
fecha_analisis = fecha_max + timedelta(days=1)
print(f"Fecha de análisis: {fecha_analisis}")

In [ ]:
# Agrupar por cliente y calcular métricas RFM
rfm = ventas.groupby('CUSTOMERNAME').agg({
    'ORDERDATE': lambda x: (fecha_analisis - x.max()).days,  # Recencia
    'ORDERNUMBER': 'nunique',  # Frecuencia
    'SALES': 'sum'  # Monetario
}).reset_index()

# Renombrar columnas
rfm.rename(columns={
    'ORDERDATE': 'Recencia',
    'ORDERNUMBER': 'Frecuencia', 
    'SALES': 'Monetario'
}, inplace=True)

# Mostrar las primeras filas
rfm.head()

In [ ]:
# Estadísticas resumidas
rfm.describe()

## 3. Puntuación RFM

In [ ]:
# Crear puntuaciones RFM con manejo robusto de errores

# Para Recencia - menor es mejor (compras más recientes)
try:
    rfm['R_Score'] = pd.qcut(rfm['Recencia'], q=5, labels=[5, 4, 3, 2, 1])
except ValueError as e:
    print(f"Advertencia para puntuación de Recencia: {e}")
    # Enfoque alternativo - usar método de rango
    rfm['R_Rank'] = rfm['Recencia'].rank(ascending=True)
    rfm['R_Score'] = pd.cut(
        rfm['R_Rank'], 
        bins=[0, rfm['R_Rank'].max()*0.2, rfm['R_Rank'].max()*0.4, 
              rfm['R_Rank'].max()*0.6, rfm['R_Rank'].max()*0.8, rfm['R_Rank'].max()], 
        labels=[5, 4, 3, 2, 1],
        include_lowest=True
    )
    
# Para Frecuencia - mayor es mejor
try:
    # Primero, verificar cuántos valores únicos tenemos
    unique_freq = rfm['Frecuencia'].nunique()
    print(f"Número de valores únicos de frecuencia: {unique_freq}")
    
    # Si tenemos valores únicos limitados, ajustar el número de cuantiles
    if unique_freq < 5:
        print(f"Advertencia: Solo {unique_freq} valores únicos de frecuencia, usando {unique_freq} cuantiles en lugar de 5")
        # Usar corte regular con bins determinados manualmente
        freq_values = sorted(rfm['Frecuencia'].unique())
        # Crear etiquetas basadas en el número de valores únicos
        freq_labels = list(range(1, unique_freq + 1))
        rfm['F_Score'] = pd.cut(rfm['Frecuencia'], bins=[-1] + freq_values, labels=freq_labels)
    else:
        rfm['F_Score'] = pd.qcut(rfm['Frecuencia'], q=5, labels=[1, 2, 3, 4, 5], duplicates='drop')
except ValueError as e:
    print(f"Advertencia para puntuación de Frecuencia: {e}")
    # Enfoque alternativo - usar método de rango
    rfm['F_Rank'] = rfm['Frecuencia'].rank(ascending=False)
    rfm['F_Score'] = pd.cut(
        rfm['F_Rank'], 
        bins=[0, rfm['F_Rank'].max()*0.2, rfm['F_Rank'].max()*0.4, 
              rfm['F_Rank'].max()*0.6, rfm['F_Rank'].max()*0.8, rfm['F_Rank'].max()], 
        labels=[1, 2, 3, 4, 5],
        include_lowest=True
    )

# Para Monetario - mayor es mejor
try:
    rfm['M_Score'] = pd.qcut(rfm['Monetario'], q=5, labels=[1, 2, 3, 4, 5])
except ValueError as e:
    print(f"Advertencia para puntuación de Monetario: {e}")
    # Enfoque alternativo - usar método de rango
    rfm['M_Rank'] = rfm['Monetario'].rank(ascending=False)
    rfm['M_Score'] = pd.cut(
        rfm['M_Rank'], 
        bins=[0, rfm['M_Rank'].max()*0.2, rfm['M_Rank'].max()*0.4, 
              rfm['M_Rank'].max()*0.6, rfm['M_Rank'].max()*0.8, rfm['M_Rank'].max()], 
        labels=[1, 2, 3, 4, 5],
        include_lowest=True
    )

# Calcular puntuación RFM global
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# Convertir a numérico para segmentación
rfm['RFM_Score_Numeric'] = rfm['R_Score'].astype(int) + rfm['F_Score'].astype(int) + rfm['M_Score'].astype(int)

# Mostrar las primeras filas con puntuaciones
rfm.head()

## 4. Segmentación de Clientes

In [ ]:
# Definir segmentos RFM
def segmento_rfm(r, f, m):
    if r >= 4 and f >= 4 and m >= 4:
        return 'Campeones'
    elif (r >= 2 and r <= 4) and (f >= 3 and f <= 4) and (m >= 3):
        return 'Clientes Leales'
    elif (r >= 3 and r <= 5) and (f >= 1 and f <= 3) and (m >= 1 and m <= 3):
        return 'Potenciales Leales'
    elif r >= 4 and (f >= 0 and f <= 1) and (m >= 0 and m <= 1):
        return 'Clientes Nuevos'
    elif (r >= 3 and r <= 4) and (f >= 0 and f <= 1) and (m >= 0 and m <= 1):
        return 'Prometedores'
    elif (r >= 2 and r <= 3) and (f >= 2 and f <= 3) and (m >= 2 and m <= 3):
        return 'Necesitan Atención'
    elif (r >= 2 and r <= 3) and (f >= 0 and f <= 2) and (m >= 0 and m <= 2):
        return 'A Punto de Dormir'
    elif r <= 2 and f >= 2 and m >= 2:
        return 'En Riesgo'
    elif r <= 1 and (f >= 4 and f <= 5) and (m >= 4 and m <= 5):
        return 'No Podemos Perderlos'
    elif (r >= 1 and r <= 2) and (f >= 1 and f <= 2) and m >= 2:
        return 'Hibernando'
    else:
        return 'Perdidos'
    
# Aplicar la función de segmentación
rfm['Segmento'] = rfm.apply(lambda x: segmento_rfm(x['R_Score'], x['F_Score'], x['M_Score']), axis=1)

# Mostrar conteos de segmentos
conteo_segmentos = rfm['Segmento'].value_counts().reset_index()
conteo_segmentos.columns = ['Segmento', 'Cantidad']
conteo_segmentos

## 5. Visualizaciones del Análisis RFM

In [ ]:
# Graficar la distribución de segmentos de clientes
plt.figure(figsize=(12, 6))
sns.barplot(x='Segmento', y='Cantidad', data=conteo_segmentos.sort_values('Cantidad', ascending=False))
plt.title('Distribución de Segmentos de Clientes', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Calcular métricas promedio por segmento
promedio_segmentos = rfm.groupby('Segmento').agg({
    'Recencia': 'mean',
    'Frecuencia': 'mean',
    'Monetario': 'mean',
    'CUSTOMERNAME': 'count'
}).reset_index()

promedio_segmentos = promedio_segmentos.rename(columns={'CUSTOMERNAME': 'Cantidad'})
promedio_segmentos = promedio_segmentos.sort_values('Cantidad', ascending=False)

# Redondear los valores para mejor legibilidad
promedio_segmentos['Recencia'] = promedio_segmentos['Recencia'].round(1)
promedio_segmentos['Frecuencia'] = promedio_segmentos['Frecuencia'].round(1)
promedio_segmentos['Monetario'] = promedio_segmentos['Monetario'].round(2)

promedio_segmentos

In [ ]:
# Crear un mapa de calor de rendimiento de segmentos
try:
    plt.figure(figsize=(12, 8))
    
    # Crear una tabla pivot adecuada
    segmento_pivot = promedio_segmentos.set_index('Segmento')[['Recencia', 'Frecuencia', 'Monetario']]
    
    # Graficar el mapa de calor
    ax = sns.heatmap(segmento_pivot, annot=True, cmap='YlGnBu', fmt='.1f')
    plt.title('Mapa de Calor de Rendimiento por Segmento', fontsize=16)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"Error al crear mapa de calor: {e}")
    print("\nAlternativa: mostrar como tabla:")
    display(promedio_segmentos[['Segmento', 'Recencia', 'Frecuencia', 'Monetario']].sort_values('Segmento'))

In [ ]:
# Visualizar el espacio RFM en 3D
fig = px.scatter_3d(rfm, x='Recencia', y='Frecuencia', z='Monetario',
                   color='Segmento', hover_name='CUSTOMERNAME',
                   opacity=0.7, width=900, height=700)
fig.update_layout(title='Segmentación RFM en 3D')
fig.show()

## 6. Análisis más Profundo por Segmento

In [ ]:
# Combinar los datos de ventas originales con los segmentos RFM
ventas_con_segmento = pd.merge(ventas, rfm[['CUSTOMERNAME', 'Segmento']], on='CUSTOMERNAME', how='left')

# Agrupar por segmento y calcular métricas
ventas_segmento = ventas_con_segmento.groupby('Segmento').agg({
    'SALES': 'sum',
    'ORDERNUMBER': 'nunique',
    'CUSTOMERNAME': 'nunique'
}).reset_index()

ventas_segmento.columns = ['Segmento', 'Ventas Totales', 'Total Pedidos', 'Cantidad Clientes']
ventas_segmento['Valor Promedio Pedido'] = ventas_segmento['Ventas Totales'] / ventas_segmento['Total Pedidos']
ventas_segmento['Valor Promedio Cliente'] = ventas_segmento['Ventas Totales'] / ventas_segmento['Cantidad Clientes']

# Redondear los valores
ventas_segmento['Valor Promedio Pedido'] = ventas_segmento['Valor Promedio Pedido'].round(2)
ventas_segmento['Valor Promedio Cliente'] = ventas_segmento['Valor Promedio Cliente'].round(2)
ventas_segmento['Ventas Totales'] = ventas_segmento['Ventas Totales'].round(2)

# Ordenar por ventas totales
ventas_segmento = ventas_segmento.sort_values('Ventas Totales', ascending=False)
ventas_segmento

In [ ]:
# Visualizar el valor promedio del cliente por segmento
plt.figure(figsize=(12, 6))
sns.barplot(x='Segmento', y='Valor Promedio Cliente', data=ventas_segmento.sort_values('Valor Promedio Cliente', ascending=False))
plt.title('Valor Promedio del Cliente por Segmento', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Visualizar el porcentaje de ventas totales por segmento
plt.figure(figsize=(10, 10))
ventas_segmento['Porcentaje Ventas'] = ventas_segmento['Ventas Totales'] / ventas_segmento['Ventas Totales'].sum() * 100
plt.pie(ventas_segmento['Porcentaje Ventas'], labels=ventas_segmento['Segmento'], autopct='%1.1f%%', 
        startangle=90, shadow=True, explode=[0.05]*len(ventas_segmento))
plt.title('Porcentaje de Ventas Totales por Segmento de Cliente', fontsize=16)
plt.axis('equal')
plt.show()

## 7. Análisis de Línea de Producto por Segmento

In [ ]:
# Analizar preferencias de producto por segmento
producto_segmento = ventas_con_segmento.groupby(['Segmento', 'PRODUCTLINE']).agg({
    'SALES': 'sum',
    'ORDERNUMBER': 'nunique'
}).reset_index()

# Ordenar por segmento y ventas
producto_segmento = producto_segmento.sort_values(['Segmento', 'SALES'], ascending=[True, False])
producto_segmento

In [ ]:
# Crear una tabla pivot para mejor visualización
producto_pivot = ventas_con_segmento.pivot_table(index='Segmento', 
                                              columns='PRODUCTLINE', 
                                              values='SALES', 
                                              aggfunc='sum')

# Visualizar las preferencias de producto
plt.figure(figsize=(14, 10))
sns.heatmap(producto_pivot, annot=True, fmt='.0f', cmap='viridis')
plt.title('Preferencias de Línea de Producto por Segmento de Cliente', fontsize=16)
plt.tight_layout()
plt.show()

## 8. Análisis Regional por Segmento

In [ ]:
# Analizar distribución regional por segmento
region_segmento = ventas_con_segmento.groupby(['Segmento', 'COUNTRY']).agg({
    'CUSTOMERNAME': 'nunique',
    'SALES': 'sum'
}).reset_index()

# Ordenar por segmento y número de clientes
region_segmento = region_segmento.sort_values(['Segmento', 'CUSTOMERNAME'], ascending=[True, False])
region_segmento.columns = ['Segmento', 'País', 'Cantidad Clientes', 'Ventas Totales']

# Obtener los 5 principales países por segmento
def get_top_n(group, n=3):
    return group.nlargest(n, 'Cantidad Clientes')

top_paises = region_segmento.groupby('Segmento').apply(get_top_n).reset_index(drop=True)
top_paises

In [ ]:
# Graficar los principales países para segmentos seleccionados
segmentos_importantes = ['Campeones', 'Clientes Leales', 'En Riesgo', 'No Podemos Perderlos']
datos_filtrados = top_paises[top_paises['Segmento'].isin(segmentos_importantes)]

plt.figure(figsize=(14, 8))
chart = sns.barplot(x='País', y='Cantidad Clientes', hue='Segmento', data=datos_filtrados)
plt.title('Principales Países por Segmentos de Clientes Importantes', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.legend(title='Segmento')
plt.tight_layout()
plt.show()

## 9. Recomendaciones para Segmentos RFM

In [ ]:
# Crear un dataframe con descripciones de segmentos y acciones recomendadas
descripciones_segmentos = pd.DataFrame({
    'Segmento': [
        'Campeones', 'Clientes Leales', 'Potenciales Leales', 'Clientes Nuevos',
        'Prometedores', 'Necesitan Atención', 'A Punto de Dormir',
        'En Riesgo', 'No Podemos Perderlos', 'Hibernando', 'Perdidos'
    ],
    'Descripción': [
        'Mejores clientes que compran con frecuencia y gastan más',
        'Compradores regulares con valor monetario superior al promedio',
        'Clientes recientes con frecuencia moderada, podrían volverse leales',
        'Clientes que recientemente hicieron sus primeras compras',
        'Compradores recientes que no han gastado mucho',
        'Valores promedio de recencia, frecuencia y monetario',
        'Recencia y frecuencia por debajo del promedio, en riesgo de pérdida',
        'Clientes que alguna vez fueron valiosos pero no han comprado recientemente',
        'Hicieron grandes compras hace tiempo, pero no han regresado recientemente',
        'Baja recencia y frecuencia, pero valor monetario superior al promedio',
        'Valores más bajos de recencia, frecuencia y monetario'
    ],
    'Acciones Recomendadas': [
        'Recompensar, vender productos premium, buscar reseñas, crear embajadores de marca',
        'Involucrar con programas de lealtad, comunicación personalizada, ofertas exclusivas',
        'Ofertas dirigidas, acceso anticipado a productos, fomentar compras más frecuentes',
        'Serie de bienvenida, educación sobre propuesta de valor, promociones para primeras compras',
        'Proporcionar incentivos para primera compra, construir relación, contenido educativo',
        'Campañas de reactivación, encuestas de satisfacción, recomendaciones personalizadas',
        'Reactivación con mayores incentivos, recordar propuesta de valor',
        'Campañas de recuperación, descuentos profundos, contenido de reenganche',
        'Contacto de reactivación, ofertas especiales de renovación, trato VIP',
        'Campañas de reactivación, recordar compras pasadas, alertas de nuevos productos',
        'Descuentos muy profundos o eliminar de lista de marketing para ahorrar costos'
    ]
})

descripciones_segmentos

## 10. Exportar Principales Clientes por Segmento

In [ ]:
# Obtener los 5 principales clientes por valor monetario en cada segmento
top_clientes = rfm.sort_values(['Segmento', 'Monetario'], ascending=[True, False])
top_clientes_segmento = top_clientes.groupby('Segmento').head(5)

# Seleccionar columnas relevantes para el informe final
top_clientes_segmento = top_clientes_segmento[['CUSTOMERNAME', 'Segmento', 'Recencia', 'Frecuencia', 'Monetario']]
top_clientes_segmento.sort_values(['Segmento', 'Monetario'], ascending=[True, False])

## 11. Resumen Ejecutivo RFM

Basado en nuestro análisis RFM, hemos segmentado la base de clientes en diferentes grupos según su comportamiento de compra. Estos son los hallazgos clave:

1. **Campeones** - Estos son nuestros mejores clientes, que compran con frecuencia y gastan más. Deben priorizarse para recompensas, ofertas especiales y programas de referencia.

2. **Clientes en Riesgo** - Hemos identificado un segmento de clientes que alguna vez fueron valiosos pero que no han comprado recientemente. Estos necesitan atención inmediata con campañas de recuperación.

3. **Potenciales Leales** - Clientes recientes con frecuencia moderada que podrían convertirse en clientes leales con el enfoque adecuado.

4. **Preferencias de Producto** - Diferentes segmentos muestran distintas preferencias de producto que pueden aprovecharse para campañas de marketing dirigidas.

5. **Distribución Regional** - Los segmentos de clientes no están distribuidos uniformemente entre regiones, lo que sugiere oportunidades para estrategias específicas por región.

**Recomendaciones:**
- Implementar estrategias de marketing específicas para cada segmento basadas en las recomendaciones proporcionadas
- Crear flujos de trabajo automatizados para mantener actualizada la información de segmentación de clientes
- Desarrollar dashboards para hacer seguimiento a la migración de segmentos a lo largo del tiempo
- Diseñar experimentos para probar la efectividad de las acciones de marketing específicas por segmento